In [262]:
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from binance.spot import Spot
from collections import namedtuple
from datetime import datetime
from operator import itemgetter
from pydash import py_

In [263]:
class ExchangeAPI(ABC):
    """Класс для работы с биржевым API."""

    @abstractmethod
    def get_historical(self):
        """Получает историю цен."""
        return

    @abstractmethod
    def get_current(self):
        """Получает текущую цену."""
        return

    @abstractmethod
    def info(self):
        """Информация об аккаунте."""
        return

In [343]:
class BinanceAPI(ExchangeAPI):
    def __init__(self, key, secret, ticker='BTCUSDT'):
        """Класс для работы с Binance API."""
        self._client = Spot(key=key, secret=secret)
        self._ticker = ticker

    def info(self):
        return self._client.account()

    def get_historical(self, interval='1m', limit=None, symbol=None):
        data = self._client.klines(interval=interval,
                                   symbol=symbol if symbol else self._ticker,
                                   limit=limit if limit else 1000)
        df = pd.DataFrame(data=map(lambda d: d[:-1],
                                   filter(lambda d: d[-1] == '0',
                                          data)),
                          columns=['open_dttm',
                                   'open',
                                   'high',
                                   'low',
                                   'close',
                                   'volume',
                                   'close_dttm',
                                   'quote_asset_volume',
                                   'n_trades',
                                   'taker_buy_base_asset_volume',
                                   'taker_buy_quote_asset_volume'])

        df = df.astype(float).assign(open_dttm=df.open_dttm.map(BinanceAPI.from_timestamp),
                                     close_dttm=df.close_dttm.map(BinanceAPI.from_timestamp)).set_index('open_dttm')

        return df

    def get_current(self):
        """TODO"""
        return

    @staticmethod
    def to_timestamp(dttm: datetime):
        return dttm.timestamp() * 1000

    @staticmethod
    def from_timestamp(ts):
        return datetime.fromtimestamp(ts / 1000)

In [344]:
%%timeit
b = BinanceAPI(**params)
client = b._client
h = b.get_historical()

1.19 s ± 342 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [345]:
class BinanceAPI(ExchangeAPI):
    def __init__(self, key, secret, ticker='BTCUSDT'):
        """Класс для работы с Binance API."""
        self._client = Spot(key=key, secret=secret)
        self._ticker = ticker

    def info(self):
        return self._client.account()

    def get_historical(self, interval='1m', limit=None, symbol=None):
        data = self._client.klines(interval=interval,
                                   symbol=symbol if symbol else self._ticker,
                                   limit=limit if limit else 1000)
        df = pd.DataFrame(data=data,
                          columns=['open_dttm',
                                   'open',
                                   'high',
                                   'low',
                                   'close',
                                   'volume',
                                   'close_dttm',
                                   'quote_asset_volume',
                                   'n_trades',
                                   'taker_buy_base_asset_volume',
                                   'taker_buy_quote_asset_volume',
                                   'ignore'])

        df = df.astype(float).assign(open_dttm=df.open_dttm.map(BinanceAPI.from_timestamp),
                                     close_dttm=df.close_dttm.map(BinanceAPI.from_timestamp)).set_index('open_dttm')

        return df

    def get_current(self):
        """TODO"""
        return

    @staticmethod
    def to_timestamp(dttm: datetime):
        return dttm.timestamp() * 1000

    @staticmethod
    def from_timestamp(ts):
        return datetime.fromtimestamp(ts / 1000)

In [346]:
params = dict(key='dXxp9qRq33vVWxmaEVcim1CC0frR7o1Nyh5N1oxm6JuCoBNABN1fb1OBZqoGgKaG',
              secret='29zKsFnbXkEXxuq3yEBPzuUwKfuOEQZ9YgpYzw9tffuFNiPZQLTtGpsbXUx7zF0A')

In [347]:
%%timeit
b = BinanceAPI(**params)
client = b._client
h = b.get_historical()

947 ms ± 235 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [303]:
h

,open,high,low,close,volume,close_dttm,quote_asset_volume,n_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
open_dttm,,,,,,,,,,
2021-11-06 07:39:00,60739.98,60746.48,60695.53,60719.93,33.12127,2021-11-06 07:39:59.999,2.011066e+06,791.0,9.92204,6.024083e+05
2021-11-06 07:40:00,60719.92,60721.00,60681.00,60689.37,19.30714,2021-11-06 07:40:59.999,1.171994e+06,721.0,4.65579,2.825970e+05
2021-11-06 07:41:00,60689.37,60770.87,60689.37,60759.09,15.34612,2021-11-06 07:41:59.999,9.320609e+05,655.0,10.46135,6.353486e+05
2021-11-06 07:42:00,60759.10,60762.86,60717.00,60746.70,28.58301,2021-11-06 07:42:59.999,1.736305e+06,748.0,14.57589,8.854093e+05
2021-11-06 07:43:00,60746.70,60752.54,60721.50,60726.16,11.53770,2021-11-06 07:43:59.999,7.007629e+05,567.0,5.18153,3.146949e+05
...,...,...,...,...,...,...,...,...,...,...
2021-11-07 00:14:00,61488.05,61520.04,61485.82,61519.22,30.18441,2021-11-07 00:14:59.999,1.856619e+06,759.0,23.25040,1.430105e+06
2021-11-07 00:15:00,61519.21,61529.06,61450.00,61479.69,16.01990,2021-11-07 00:15:59.999,9.848640e+05,787.0,8.22105,5.054022e+05
2021-11-07 00:16:00,61479.69,61499.99,61466.83,61481.66,13.95952,2021-11-07 00:16:59.999,8.582493e+05,446.0,9.33170,5.736934e+05


In [201]:
client.time()

{'serverTime': 1636242384421}

In [69]:
ticker = 'BTCUSDT'

In [70]:
client.ticker_price(ticker)

{'symbol': 'BTCUSDT', 'price': '61386.70000000'}

In [75]:
client.exchange_info(ticker)

{'timezone': 'UTC',
 'serverTime': 1636239823021,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 1200},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 50},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 160000},
  {'rateLimitType': 'RAW_REQUESTS',
   'interval': 'MINUTE',
   'intervalNum': 5,
   'limit': 6100}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'BTCUSDT',
   'status': 'TRADING',
   'baseAsset': 'BTC',
   'baseAssetPrecision': 8,
   'quoteAsset': 'USDT',
   'quotePrecision': 8,
   'quoteAssetPrecision': 8,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 8,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'isSpotTradingAllowed': True,
   'isMarginTradingAllowed': True

In [76]:
client.coin_info()

[{'coin': 'AGLD',
  'depositAllEnable': True,
  'withdrawAllEnable': True,
  'name': 'Adventure Gold',
  'free': '0',
  'locked': '0',
  'freeze': '0',
  'withdrawing': '0',
  'ipoing': '0',
  'ipoable': '0',
  'storage': '0',
  'isLegalMoney': False,
  'trading': True,
  'networkList': [{'network': 'ETH',
    'coin': 'AGLD',
    'withdrawIntegerMultiple': '0.00000001',
    'isDefault': True,
    'depositEnable': True,
    'withdrawEnable': True,
    'depositDesc': '',
    'withdrawDesc': '',
    'specialTips': '',
    'specialWithdrawTips': '',
    'name': 'Ethereum (ERC20)',
    'resetAddressStatus': False,
    'addressRegex': '^(0x)[0-9A-Fa-f]{40}$',
    'memoRegex': '',
    'withdrawFee': '13',
    'withdrawMin': '26',
    'withdrawMax': '9999999',
    'minConfirm': 12,
    'unLockConfirm': 0,
    'sameAddress': False}]},
 {'coin': 'STPT',
  'depositAllEnable': True,
  'withdrawAllEnable': True,
  'name': 'Standard Tokenization Protocol',
  'free': '0',
  'locked': '0',
  'freeze':

In [156]:
label_data(k)

{'open_dttm': 1636241160000,
 'open': '61395.64000000',
 'high': '61408.07000000',
 'low': '61388.25000000',
 'close': '61400.93000000',
 'volume': '6.44457000',
 'close_dttm': 1636241219999,
 'quote_asset_volume': '395674.31873870',
 'n_trades': 379,
 'taker_buy_base_asset_volume': '2.94665000',
 'taker_buy_quote_asset_volume': '180916.69586480',
 'ignore': '0'}

In [157]:
k = client.klines('BTCUSDT', interval='1m', limit=3)
k

[[1636241400000,
  '61396.90000000',
  '61409.77000000',
  '61394.13000000',
  '61407.16000000',
  '10.72082000',
  1636241459999,
  '658300.28240630',
  383,
  '6.60842000',
  '405779.45657730',
  '0'],
 [1636241460000,
  '61407.17000000',
  '61409.99000000',
  '61398.44000000',
  '61402.94000000',
  '13.96139000',
  1636241519999,
  '857278.13619340',
  441,
  '7.31295000',
  '449039.95902040',
  '0'],
 [1636241520000,
  '61402.94000000',
  '61408.11000000',
  '61397.46000000',
  '61397.47000000',
  '7.03516000',
  1636241579999,
  '431976.75429440',
  223,
  '1.92497000',
  '118200.31441910',
  '0']]

,0,1,2,3,4,5,6,7,8,9,10,11
0,1636241400000,61396.90000000,61409.77000000,61394.13000000,61407.16000000,10.72082000,1636241459999,658300.28240630,383,6.60842000,405779.45657730,0
1,1636241460000,61407.17000000,61409.99000000,61398.44000000,61402.94000000,13.96139000,1636241519999,857278.13619340,441,7.31295000,449039.95902040,0
2,1636241520000,61402.94000000,61408.11000000,61397.46000000,61397.47000000,7.03516000,1636241579999,431976.75429440,223,1.92497000,118200.31441910,0


In [134]:
[*map(BinanceAPI.from_timestamp, k)]

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [127]:
BinanceAPI.from_timestamp(t)

datetime.datetime(2021, 11, 6, 23, 22)